# ETL Pipeline

This notebook will use disaster response message data from [Figure 8](https://www.figure-eight.com/) to create an ETL pipeline.

In [1]:
import pandas as pd
import numpy as np
import sqlite3

## Assess

### Messages

In [2]:
messages = pd.read_csv('disaster_messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [6]:
messages.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

**Oberservations:** 
- It appears that the messages are available in both English and French. Because we will only be cleaning the data in English, it is possible to drop the `original` column.
- If the `genre` column is to be used, it will need to be converted into dummy variables. It seems that this column could also be considered the "source".

### Categories

In [3]:
categories = pd.read_csv('disaster_categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [4]:
# Review contents of categories column
print(categories.categories.iloc[0])

related-1;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0


How many categories are there?

In [5]:
len(categories.categories.iloc[0].split(';'))

36

**Observations:**
- To prep for machine learning the categories column needs to be split into 36 columns
- Once the categories have been joined to the messages, it would be possible to drop the id column

## Clean

In [8]:
messages_clean = messages.copy()
categories_clean = categories.copy()

### Categories

**1. Create new df of categories with category name for heading and 1s and 0s for values**

- Use `.str.split(';', expand=True)` to split data
- Use `.str[:-2]` to get headings
- Use `.str[-1]` and `.astype(int)` to convert data to 1s and 0s

In [9]:
# Create df
df = categories_clean.categories.str.split(';', expand=True)
# Confirm 36 columns created
assert df.shape[1] == 36, "df columns: {}".format(df.shape[1])

In [10]:
# Get headings
col_names = df.iloc[0].str[:-2]
df.columns = col_names
df.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object', name=0)

In [12]:
# Convert columns to int
for col in col_names:
    df[col] = df[col].str[-1].astype(int)

# Confirm 36 int columns
assert (df.dtypes == int).sum() == 36, df.dtypes

In [14]:
# Confirm that all messages have only one category
assert df.sum(axis=1).sum() == df.shape[0], df.sum(axis=1).sum()

AssertionError: 83434

In [15]:
df[df.sum(axis=1) != 1]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10,1,1,0,1,1,1,0,0,0,0,...,0,1,1,1,0,0,0,0,0,1
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


**Note:** Messages can be categorized as more than one type so category prediction will need to manage this. 

**2. Replace `categories` column with df**
- Use `pd.concat([categories_clean, df], axis=1)` & `.drop('categories', axis=1)`

In [16]:
categories_clean = pd.concat([categories_clean.drop('categories', axis=1), df], axis=1)
# Confirm total of 37 columns
assert categories_clean.shape[1] == 37, "Categories shape: {}".format(categories_clean.shape[1])

**3. Create merged dataset**
- Use `.merge(on='id')`

The `original` column and `id` column were retained for storage in the database even if these will not be used in the machine learning process.

In [25]:
df_merged = messages_clean.merge(categories_clean, on='id')
# Confirm correct new shape
assert df_merged.shape[1] == messages_clean.shape[1] + categories_clean.shape[1] - 1, \
    "Merged df columns: {}, Combined columns: {}".format(df_merged.shape[1], 
                                                         messages_clean.shape[1] + categories_clean.shape[1] - 1)
assert df_merged.shape[0] == messages_clean.shape[0], "Merged df rows: {}, Messages df rows: {}"\
    .format(df_merged.shape[0], messages_clean.shape[0])

AssertionError: Merged df rows: 26386, Messages df rows: 26248

It seems like joining caused some duplicate rows, so these need to be removed.

Let's have a look at the duplicate rows.

In [24]:
df_merged[df_merged.duplicated()]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
164,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
165,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
658,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
659,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
660,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
716,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1415,1652,"please we need water, food and tents, we have ...","p jwen dlo, manje, tant pou nou demi nou gen 1...",direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1416,1652,"please we need water, food and tents, we have ...","p jwen dlo, manje, tant pou nou demi nou gen 1...",direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2139,2446,How much money did TV Latino American collect ...,Konbyen kob tv latino america ranmase miami,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


There's 170 duplicated rows that need to be dropped. However, this is more than the difference between the rows - it seems there might have been some duplicates in the original data.


What's also interesting is that some of the values are not 1 or 0, so that should also be explored. 

**4. Drop duplicated rows.**
- Use `.drop_duplicates(inplace=True)`

In [26]:
df_merged.drop_duplicates(inplace=True)
# Confirm no duplicates
assert df_merged.duplicated().sum() == 0

**5. Explore values for categories**

In [28]:
for col in df_merged.columns[4:]:
    if (df_merged[col] > 1).sum() > 0:
        print(col)
        print(df_merged[col].unique())
        print('\n')

related
[1 0 2]




It's just the `related` column that has these three different values. For the purposes of simply predicting related or not it is preferrable to only have "on/off" conditions. As a result, any value of 2 should be converted to 1.

In [31]:
df_merged.loc[(df_merged.related == 2), 'related'] = 1
# Confirm only two values
assert len(df_merged.related.unique()) == 2, df.related.unique()

## Load

In [32]:
conn = sqlite3.connect('disaster_messages.db')
df_merged.to_sql('messages', con=conn, if_exists='replace', index=False)
conn.commit()
conn.close()